In [24]:
import numpy as np
from IPython.display import HTML
import time
import pandas as pd
import math
import ipywidgets as widgets
from ipywidgets import interact, fixed, interact_manual
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from matplotlib import rc
rc('animation', html='jshtml')


In [10]:
class Timer:
    def __init__(self, msg='Time elapsed'):
        self.msg = msg
    def __enter__(self):
        self.start = time.time()
        return self
    def __exit__(self, *args):
        self.end = time.time()
        duration = self.end - self.start
        print(f'{self.msg}: {duration:.2f}s')

class Event:
    __slots__ = 't', 'x', 'y', 'p'
    def __init__(self, t, x, y, p):
        self.t = t
        self.x = x
        self.y = y
        self.p = p
    def __repr__(self):
        return f'Event(t={self.t:.3f}, x={self.x}, y={self.y}, p={self.p})'


In [40]:
def normalize_image(image, percentile_lower=1, percentile_upper=99):
    mini, maxi = np.percentile(image, (percentile_lower, percentile_upper))
    if mini == maxi:
        return 0 * image + 0.5  # gray image
    return np.clip((image - mini) / (maxi - mini + 1e-5), 0, 1)


def animate(images):
    fig = plt.figure()
    ims = []
    for image in images:
        im = plt.imshow(normalize_image(image), cmap='gray', vmin=0, vmax=1, animated=True)
        ims.append([im])
    ani = ArtistAnimation(fig, ims, interval=50, blit=False, repeat_delay=1000)
    plt.close(ani._fig)
    return ani


def high_pass_filter(events, cutoff_frequency=5):
    print('Reconstructing, please wait...')
    events_per_frame = 2e4
    with Timer('Reconstruction'):
        time_surface = np.zeros((height, width), dtype=np.float32)
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            beta = math.exp(-cutoff_frequency * (e.t - time_surface[e.y, e.x]))
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            time_surface[e.y, e.x] = e.t
            if i % events_per_frame == 0:
                beta = np.exp(-cutoff_frequency * (e.t - time_surface))
                image_state *= beta
                time_surface.fill(e.t)
                image_list.append(np.copy(image_state))
    return animate(image_list)

def high_pass_filter_simple(events, beta=0.8):
    print('Reconstructing, please wait...')
    events_per_frame = 2e4
    with Timer('Reconstruction (simple)'):
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            if i % events_per_frame == 0:
                image_list.append(np.copy(image_state))
    return animate(image_list)


In [39]:
with Timer('Loading'):
    n_events = 2e6
    path_to_events = 'data/boxes_6dof/events.zip'
    header = pd.read_csv(path_to_events, delim_whitespace=True, names=['width', 'height'],
                         dtype={'width': np.int, 'height': np.int}, nrows=1)
    width, height = header.values[0]
    print(f'width, height: {width}, {height}')
    event_pd = pd.read_csv(path_to_events, delim_whitespace=True, header=None,
                              names=['t', 'x', 'y', 'p'],
                              dtype={'t': np.float64, 'x': np.int16, 'y': np.int16, 'p': np.int8},
                              engine='c', skiprows=1, nrows=n_events, memory_map=True)
    event_list = []
    for event in event_pd.values:
        t, x, y, p = event
        event_list.append(Event(t, int(x), int(y), -1 if p < 0.5 else 1))
    print('Loaded {:.2f}M events'.format(len(event_list) / 1e6))
        

width, height: 240, 180
Loaded 2.00M events
Loading: 4.81s


Click ```Run Interact``` then wait for the animation to appear to see the result.

Use the slider then click ```Run Interact``` again to try different settings.

In [45]:
interact_manual(high_pass_filter, events=fixed(event_list), cutoff_frequency=(0, 30, 0.01));

interactive(children=(FloatSlider(value=5.0, description='cutoff_frequency', max=30.0, step=0.01), Button(desc…

Set ```beta = 1``` for direct integration.

In [43]:
interact_manual(high_pass_filter_simple, events=fixed(event_list), beta=(0, 1, 0.01));

interactive(children=(FloatSlider(value=0.8, description='beta', max=1.0, step=0.01), Button(description='Run …